In [1]:
import sys
sys.path.append('../src/')
from dashboard.utils.data import *
from dashboard.utils.plotting import *
%load_ext autoreload
%autoreload 2

In [2]:
data = load_synthetic_data.__wrapped__()

In [3]:
data.columns

Index(['Load', 'Hydro Load Factor', 'Hydro Reservoir Obs',
       'Hydro RoR Load Factor', 'Hydro RoR Obs', 'Solar Photo Load Factor',
       'Solar Photo Obs', 'Wind Offshore Load Factor', 'Wind Offshore Obs',
       'Wind Onshore Load Factor', 'Wind Onshore Obs', 'Temperature',
       'Irradiance', 'Sea Level Pressure', 'Precipitation', 'Wind Speed (10m)',
       'Wind Speed (100m)', 'Total Wind Obs', 'Total Wind Capacity',
       'Wind Load Factor', 'Solar Load Factor', 'Total Solar Capacity',
       'Total Solar Obs'],
      dtype='object')

In [4]:
def get_correlation(data, args, start_date = None, end_date = None, season = None, frequency = None, agg_fn = None):
    multi_index = []
    values = []
    for dict_ in args:
        multi_index.append((dict_['country'], dict_['variable']))
        col = data.loc[(dict_['country'], slice(start_date, end_date)), dict_['variable']]
        if frequency:
            col = col.xs(dict_['country'], level = 0).resample(frequency).agg(agg_fn)
        if season:
            col = col.loc[col.index.month.isin(season)]
        values.append(col)
    
    corr_df = pd.DataFrame(
        index = pd.MultiIndex.from_tuples(multi_index),
        columns = pd.MultiIndex.from_tuples(multi_index)
    )
    index_comb, val_comb = list(itertools.combinations(multi_index,2)), list(itertools.combinations(values, 2))
                               
    for i_comb, v_comb in zip(index_comb, val_comb):
        i_A, i_B = i_comb
        
        
        val_A, val_B = v_comb

        assert len(val_A) == len(val_B), "Mispecified correlation between %s (%s) and %s (%s): the two columns are specified with a different length (%d and %d)" %(*i_A[::-1], *i_B[::-1], len(val_A), len(val_B))
        mask = (~np.ma.masked_invalid(val_A).mask & ~np.ma.masked_invalid(val_B).mask)
        corr_df.loc[i_A, i_B] = corr_df.loc[i_B, i_A] = np.ma.corrcoef(val_A[mask], val_B[mask])[0,1]
    
    for idx in corr_df.index:
        corr_df.loc[idx, idx] = 1
    
    def style_func(val):
        if val == 1:
            return "color:black"
        elif val < 0:
            return "color:red" if val > -0.7 else "background-color:red; color:white"
        elif val > 0:
            return 'color:blue' if val < 0.7 else "background-color:blue; color:white"
        
    return corr_df.style.applymap(style_func)

## Monthly Correlation

In [5]:
args = [{"country": "GE", "variable": "Wind Load Factor"},
        {"country": "GE", "variable": "Load"},
        {"country": "GE", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [6]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [7]:
args = [{"country": "GE", "variable": "Wind Speed (100m)"},
        {"country": "GE", "variable": "Temperature"},
        {"country": "GE", "variable": "Irradiance"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [10]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [11]:
args = [{"country": "FR", "variable": "Wind Load Factor"},
        {"country": "FR", "variable": "Load"},
        {"country": "FR", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [12]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [13]:
args = [{"country": "FR", "variable": "Wind Speed (100m)"},
        {"country": "FR", "variable": "Temperature"},
        {"country": "FR", "variable": "Irradiance"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [14]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [15]:
args = [{"country": "UK", "variable": "Wind Offshore Load Factor"},
        {"country": "UK", "variable": "Wind Onshore Load Factor"},
        {"country": "UK", "variable": "Load"},
        {"country": "UK", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [16]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [17]:
args = [{"country": "UK", "variable": "Wind Speed (100m)"},
        {"country": "UK", "variable": "Temperature"},
        {"country": "UK", "variable": "Irradiance"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [18]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [19]:
args = [{"country": "ES", "variable": "Wind Load Factor"},
        {"country": "ES", "variable": "Load"},
        {"country": "ES", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [20]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

In [21]:
args = [{"country": "ES", "variable": "Wind Speed (100m)"},
        {"country": "ES", "variable": "Temperature"},
        {"country": "ES", "variable": "Irradiance"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [22]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

## Daily Correlation

In [82]:
args = [{"country": "GE", "variable": "Wind Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "GE", "variable": "Load", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "GE", "variable": "Solar Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "GE", "variable": "Wind Speed (100m)", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "GE", "variable": "Irradiance", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "GE", "variable": "Temperature", "start_date": None, "end_date": None, "frequency": None, "agg": None}]
get_correlation(data, args)

In [83]:
args = [{"country": "FR", "variable": "Wind Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "FR", "variable": "Load", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "FR", "variable": "Solar Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "FR", "variable": "Wind Speed (100m)", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "FR", "variable": "Irradiance", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "FR", "variable": "Temperature", "start_date": None, "end_date": None, "frequency": None, "agg": None}]
get_correlation(data, args)

In [84]:
args = [{"country": "UK", "variable": "Wind Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "UK", "variable": "Load", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "UK", "variable": "Solar Load Factor", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "UK", "variable": "Wind Speed (100m)", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "UK", "variable": "Irradiance", "start_date": None, "end_date": None, "frequency": None, "agg": None},
        {"country": "UK", "variable": "Temperature", "start_date": None, "end_date": None, "frequency": None, "agg": None}]
get_correlation(data, args)

In [119]:
args = [{"country": "ES", "variable": "Wind Speed (100m)"},
        {"country": "ES", "variable": "Wind Load Factor"},
        {"country": "ES", "variable": "Temperature"},
        {"country": "ES", "variable": "Load"},
        {"country": "ES", "variable": "Irradiance"},
        {"country": "ES", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [12,1,2], frequency = 'D', agg_fn = 'mean')

In [118]:
get_correlation(data, args, season = [6,7,8], frequency = 'D', agg_fn = 'mean')

## Cross-Country Correlation

In [23]:
args = [{"country": "GE", "variable": "Wind Load Factor"},
        {"country": "FR", "variable": "Wind Load Factor"},
        {"country": "ES", "variable": "Wind Load Factor"},
        {"country": "UK", "variable": "Wind Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Wind Load Factor,Wind Load Factor,Wind Load Factor,Wind Load Factor
GE,Wind Load Factor,1,0.673817,-0.189459,0.728236
FR,Wind Load Factor,0.673817,1,0.376445,0.480172
ES,Wind Load Factor,-0.189459,0.376445,1,-0.192067
UK,Wind Load Factor,0.728236,0.480172,-0.192067,1


In [24]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Wind Load Factor,Wind Load Factor,Wind Load Factor,Wind Load Factor
GE,Wind Load Factor,1,0.700772,0.396960,0.414167
FR,Wind Load Factor,0.700772,1,0.606059,0.263174
ES,Wind Load Factor,0.396960,0.606059,1,0.154111
UK,Wind Load Factor,0.414167,0.263174,0.154111,1


In [25]:
args = [{"country": "GE", "variable": "Solar Load Factor"},
        {"country": "FR", "variable": "Solar Load Factor"},
        {"country": "ES", "variable": "Solar Load Factor"},
        {"country": "UK", "variable": "Solar Load Factor"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Solar Load Factor,Solar Load Factor,Solar Load Factor,Solar Load Factor
GE,Solar Load Factor,1,0.874766,0.572757,0.830222
FR,Solar Load Factor,0.874766,1,0.714050,0.780662
ES,Solar Load Factor,0.572757,0.714050,1,0.649611
UK,Solar Load Factor,0.830222,0.780662,0.649611,1


In [26]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Solar Load Factor,Solar Load Factor,Solar Load Factor,Solar Load Factor
GE,Solar Load Factor,1,0.621092,0.159031,0.436237
FR,Solar Load Factor,0.621092,1,0.465072,0.330277
ES,Solar Load Factor,0.159031,0.465072,1,-0.184333
UK,Solar Load Factor,0.436237,0.330277,-0.184333,1


In [27]:
args = [{"country": "GE", "variable": "Load"},
        {"country": "FR", "variable": "Load"},
        {"country": "ES", "variable": "Load"},
        {"country": "UK", "variable": "Load"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Load,Load,Load,Load
GE,Load,1,0.656599,0.664869,0.769785
FR,Load,0.656599,1,0.727829,0.709805
ES,Load,0.664869,0.727829,1,0.538645
UK,Load,0.769785,0.709805,0.538645,1


In [28]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Load,Load,Load,Load
GE,Load,1,0.688028,0.456539,0.406149
FR,Load,0.688028,1,0.421023,0.678087
ES,Load,0.456539,0.421023,1,-0.131447
UK,Load,0.406149,0.678087,-0.131447,1


In [29]:
args = [{"country": "GE", "variable": "Temperature"},
        {"country": "FR", "variable": "Temperature"},
        {"country": "ES", "variable": "Temperature"},
        {"country": "UK", "variable": "Temperature"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Temperature,Temperature,Temperature,Temperature
GE,Temperature,1,0.849548,0.458355,0.817707
FR,Temperature,0.849548,1,0.756258,0.677555
ES,Temperature,0.458355,0.756258,1,0.316904
UK,Temperature,0.817707,0.677555,0.316904,1


In [30]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')

,,GE,FR,ES,UK
,,Temperature,Temperature,Temperature,Temperature
GE,Temperature,1,0.866910,0.610535,0.776361
FR,Temperature,0.866910,1,0.822184,0.827978
ES,Temperature,0.610535,0.822184,1,0.628615
UK,Temperature,0.776361,0.827978,0.628615,1


In [32]:
args = [{"country": "GE", "variable": "Wind Load Factor"},
        {"country": "GE", "variable": "Solar Load Factor"},
        {"country": "GE", "variable": "Load"},
        {"country": "FR", "variable": "Wind Load Factor"},
        {"country": "FR", "variable": "Solar Load Factor"},
        {"country": "FR", "variable": "Load"},
        {"country": "ES", "variable": "Wind Load Factor"},
        {"country": "ES", "variable": "Solar Load Factor"},
        {"country": "ES", "variable": "Load"},
        {"country": "UK", "variable": "Wind Load Factor"},
        {"country": "UK", "variable": "Solar Load Factor"},
        {"country": "UK", "variable": "Load"}]
get_correlation(data, args, season = [1,2,12], frequency = 'M', agg_fn = 'mean')

In [33]:
get_correlation(data, args, season = [6,7,8], frequency = 'M', agg_fn = 'mean')